# Machine Learning Modeling

**Description:** 
This notebook contains the code for building and evaluating machine learning models to predict diabetes and hypertension prevalence based on PM2.5 levels and other features. The models are trained with control for confounding variables and include interaction terms to assess the impact of outside factors like urbanization and socioeconomic status.

## 1. Setup and Data Preparation

In this section, we will import necessary libraries, load the dataset, and perform data processing and normalization.

In [1]:
import os, csv
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from utils import build_dataset

In [2]:
# Load the dataset
data = build_dataset()

# Select features and targets to be used in modeling
features = ["avg_pm25","smoking","obesity","no_lt_physical_activity","binge_drinking","lack_of_health_insurance","routine_checkup","food_insecurity","housing_insecurity","urbanization_level"]
targets = ["diabetes","hypertension"]

# Split the dataset into features and target variables
X = data[features]
y = data[targets]

In [3]:
# Split the dataset into features and target variables for evaluating model performance
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1624, 10), (407, 10), (1624, 2), (407, 2))

In [4]:
X_train.head()

,avg_pm25,smoking,obesity,no_lt_physical_activity,binge_drinking,lack_of_health_insurance,routine_checkup,food_insecurity,housing_insecurity,urbanization_level
1683,6.549546,16.704815,37.383491,27.787189,18.058957,17.555141,73.811043,14.930126,12.819815,6.0
471,8.458216,13.100000,35.700000,23.300000,21.700000,6.100000,79.100000,7.200000,7.100000,6.0
744,6.526506,19.896181,39.260925,25.995817,19.140221,7.028860,76.929655,15.857115,13.280515,5.0
1111,6.826042,14.800000,38.000000,26.200000,18.000000,8.200000,76.000000,10.400000,8.800000,6.0
1481,8.826063,20.821889,42.944760,31.337431,14.229319,14.886096,80.701485,23.386609,18.745315,2.0


In [5]:
y_train.head()

,diabetes,hypertension
1683,13.702892,37.099061
471,10.600000,31.600000
744,12.092917,36.131869
1111,12.100000,39.100000
1481,16.337077,41.789732


Normalization is done by subtracting the mean and dividing by the standard deviation of feature values where the formula is defined as:
$$
{z} = \frac{x_{i,j} - \tilde{x}_{j}}{sd(x_{j})}
$$


In [6]:
# Normalize the numeric features (features except "urbanization_level" which is categorical) on X
scaler = StandardScaler()

numeric_features = features.copy()
numeric_features.remove("urbanization_level")

X_scaled = X.copy()
X_scaled[numeric_features] = scaler.fit_transform(X[numeric_features])

In [7]:
# Normalize the numeric features on X_train and X_test
scaler = StandardScaler()

X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

# Fit the scaler on the training data and normalize both training and test data
X_train_scaled[numeric_features] = scaler.fit_transform(X_train[numeric_features])
X_test_scaled[numeric_features] = scaler.transform(X_test[numeric_features])

In [8]:
X_train_scaled.head()

,avg_pm25,smoking,obesity,no_lt_physical_activity,binge_drinking,lack_of_health_insurance,routine_checkup,food_insecurity,housing_insecurity,urbanization_level
1683,-1.229986,-0.274834,-0.203155,-0.083074,0.664311,0.958623,-0.725213,-0.192270,-0.104958,6.0
471,0.198062,-1.229711,-0.570384,-0.918064,2.054788,-0.974968,0.778932,-1.438531,-1.441306,6.0
744,-1.247225,0.570526,0.206379,-0.416418,1.077235,-0.818179,0.161700,-0.042820,0.002678,5.0
1111,-1.023115,-0.779399,-0.068673,-0.378423,0.641796,-0.620494,-0.102688,-0.922623,-1.044127,6.0
1481,0.473281,0.815736,1.009954,0.577566,-0.798187,0.508097,1.234384,1.171095,1.279446,2.0


## 2. Linear Regression Model for PM2.5 and Diabetes/Hypertension Prevalence

To test the main hypothesis that higher PM2.5 levels are associated with increased prevalence of diabetes and hypertension, we will build regression models controlling for confounding variables.

### 2.1 - OLS Regression Model with Interaction Terms

We will build two separate OLS regression models: one for diabetes prevalence and another for hypertension prevalence. Both models will include interaction terms to assess how the relationship between PM2.5 and health outcomes varies by different confounding factors.

In [9]:
# Statsmodels OLS Regression with Interaction Terms
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [10]:
formula_diabetes = "diabetes ~ avg_pm25" \
    " + smoking" \
    " + obesity" \
    " + no_lt_physical_activity" \
    " + binge_drinking" \
    " + lack_of_health_insurance" \
    " + routine_checkup" \
    " + food_insecurity" \
    " + housing_insecurity" \
    " + C(urbanization_level)"

formula_hypertension = "hypertension ~ avg_pm25" \
    " + smoking" \
    " + obesity" \
    " + no_lt_physical_activity" \
    " + binge_drinking" \
    " + lack_of_health_insurance" \
    " + routine_checkup" \
    " + food_insecurity" \
    " + housing_insecurity" \
    " + C(urbanization_level)"

In [11]:
model_diabetes = smf.ols(formula=formula_diabetes, data=pd.concat([X_scaled, y], axis=1)).fit()
model_hypertension = smf.ols(formula=formula_hypertension, data=pd.concat([X_scaled, y], axis=1)).fit()
model_diabetes.mse_resid, model_hypertension.mse_resid

(np.float64(0.9356413793708996), np.float64(5.14069068251815))

In [12]:
model_diabetes.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               diabetes   R-squared:                       0.877
Model:                            OLS   Adj. R-squared:                  0.876
Method:                 Least Squares   F-statistic:                     1029.
Date:                Fri, 02 Jan 2026   Prob (F-statistic):               0.00
Time:                        23:56:34   Log-Likelihood:                -2806.8
No. Observations:                2031   AIC:                             5644.
Df Residuals:                    2016   BIC:                             5728.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                       13.7995      0.173     79.718      0.000      13.460      14.139
C(urbanization_level)[T.2.0]    -0.2628      0.183     -1.434      0.152      -0.622       0.097
C(urbanization_level)[T.3.0]    -0.2889      0.181     -1.593      0.111      -0.645       0.067
C(urbanization_level)[T.4.0]    -0.2965      0.183     -1.620      0.105      -0.655       0.062
C(urbanization_level)[T.5.0]    -0.3151      0.179     -1.760      0.079      -0.666       0.036
C(urbanization_level)[T.6.0]    -0.0615      0.179     -0.344      0.731      -0.412       0.289
avg_pm25                        -0.1142      0.024     -4.818      0.000      -0.161      -0.068
smoking                          0.2128      0.047      4.550      0.000       0.121       0.305
obesity                          0.2213      0.037      5.998      0.000       0.149       0.294
no_lt_physical_activity          0.9644      0.070     13.787      0.000       0.827       1.102
binge_drinking                  -0.5202      0.031    -16.705      0.000      -0.581      -0.459
lack_of_health_insurance         0.4642      0.044     10.561      0.000       0.378       0.550
routine_checkup                  0.5790      0.029     19.711      0.000       0.521       0.637
food_insecurity                  0.9582      0.158      6.046      0.000       0.647       1.269
housing_insecurity              -0.4591      0.139     -3.314      0.001      -0.731      -0.187
==============================================================================
Omnibus:                       78.572   Durbin-Watson:                   1.164
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              219.998
Skew:                          -0.101   Prob(JB):                     1.69e-48
Kurtosis:                       4.600   Cond. No.                         42.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
model_hypertension.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           hypertension   R-squared:                       0.836
Model:                            OLS   Adj. R-squared:                  0.835
Method:                 Least Squares   F-statistic:                     733.8
Date:                Fri, 02 Jan 2026   Prob (F-statistic):               0.00
Time:                        23:56:34   Log-Likelihood:                -4536.9
No. Observations:                2031   AIC:                             9104.
Df Residuals:                    2016   BIC:                             9188.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                       36.3399      0.406     89.562      0.000      35.544      37.136
C(urbanization_level)[T.2.0]     1.0386      0.430      2.418      0.016       0.196       1.881
C(urbanization_level)[T.3.0]     1.1583      0.425      2.725      0.006       0.325       1.992
C(urbanization_level)[T.4.0]     1.1214      0.429      2.614      0.009       0.280       1.963
C(urbanization_level)[T.5.0]     1.2299      0.420      2.930      0.003       0.407       2.053
C(urbanization_level)[T.6.0]     1.8785      0.419      4.489      0.000       1.058       2.699
avg_pm25                        -0.2483      0.056     -4.469      0.000      -0.357      -0.139
smoking                          1.3155      0.110     12.000      0.000       1.100       1.530
obesity                          0.7075      0.086      8.181      0.000       0.538       0.877
no_lt_physical_activity          0.8370      0.164      5.105      0.000       0.515       1.159
binge_drinking                  -2.1317      0.073    -29.207      0.000      -2.275      -1.989
lack_of_health_insurance         0.2355      0.103      2.285      0.022       0.033       0.437
routine_checkup                  1.8394      0.069     26.716      0.000       1.704       1.974
food_insecurity                 -1.2564      0.371     -3.382      0.001      -1.985      -0.528
housing_insecurity               1.1616      0.325      3.577      0.000       0.525       1.798
==============================================================================
Omnibus:                       23.518   Durbin-Watson:                   1.273
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               27.546
Skew:                          -0.194   Prob(JB):                     1.04e-06
Kurtosis:                       3.418   Cond. No.                         42.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**Interpretation of Coefficients:**
- The negative coefficient for `avg_pm25` indicates that there is an inverse relationship between PM2.5 levels and diabetes prevalence when controlling for other factors. This contradiction with our initial hypothesis suggests that there may be collinearity or other confounding effects at play.

To further investigate, we will examine the covariance of features to identify potential multicollinearity issues.

### 1.2 - Multicollinearity Check

In this section, we will use Variance Inflation Factor (VIF) to check for multicollinearity among the features used in the regression models. High VIF values indicate that a feature is highly collinear with other features, which can distort the regression coefficients and their interpretations.

In [14]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

X_vif = X
# add a constant for VIF calculation
X_vif = sm.add_constant(X_vif)

vif_data = pd.DataFrame()
vif_data["feature"] = X_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]

vif_data

,feature,VIF
0,const,1029.527001
1,avg_pm25,1.210696
2,smoking,4.731740
3,obesity,2.897279
4,no_lt_physical_activity,10.561415
5,binge_drinking,2.095132
6,lack_of_health_insurance,4.165971
7,routine_checkup,1.862111
8,food_insecurity,54.431913
9,housing_insecurity,41.642344


The results of the VIF analysis show that no_lt_physical_activity, food_insecurity and housing_insecurity have high VIF values, indicating potential multicollinearity issues. In order to address this, we will combine these features into a single composite variable called "lifestyle_insecurity" by taking their average.

In [15]:
X_scaled["lifestyle_insecurity"] = X_scaled[["no_lt_physical_activity", "food_insecurity", "housing_insecurity"]].mean(axis=1)
# Drop the original features
X_scaled = X_scaled.drop(columns=["no_lt_physical_activity", "food_insecurity", "housing_insecurity"])

# Scale the new feature
scaler = StandardScaler()
X_scaled["lifestyle_insecurity"] = scaler.fit_transform(X_scaled[["lifestyle_insecurity"]])

X_scaled[["lifestyle_insecurity"]].head()

,lifestyle_insecurity
0,-0.240699
1,-0.548915
2,2.016696
3,0.904355
4,0.015868


In [16]:
# Redo VIF calculation after combining features
X_vif = X_scaled
# add a constant for VIF calculation
X_vif = sm.add_constant(X_vif)

vif_data = pd.DataFrame()
vif_data["feature"] = X_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]

vif_data

,feature,VIF
0,const,12.370022
1,avg_pm25,1.209461
2,smoking,2.946046
3,obesity,2.480474
4,binge_drinking,1.836951
5,lack_of_health_insurance,3.718259
6,routine_checkup,1.713318
7,urbanization_level,1.139615
8,lifestyle_insecurity,6.779524


The constructed composite lifestyle insecurity index addressed multicollinearity among socioeconomic variables. Variance inflation factors in the final models were all below 7, indicating acceptable levels of collinearity.

A revised OLS regression model will be built using the new composite variable to reassess the relationship between PM2.5 and health outcomes.

In [17]:
formula_diabetes_revised = "diabetes ~ avg_pm25" \
    " + smoking" \
    " + obesity" \
    " + binge_drinking" \
    " + lack_of_health_insurance" \
    " + routine_checkup" \
    " + lifestyle_insecurity" \
    " + C(urbanization_level)"

formula_hypertension_revised = "hypertension ~ avg_pm25" \
    " + smoking" \
    " + obesity" \
    " + binge_drinking" \
    " + lack_of_health_insurance" \
    " + routine_checkup" \
    " + lifestyle_insecurity" \
    " + C(urbanization_level)"

model_diabetes_revised = smf.ols(formula=formula_diabetes_revised, data=pd.concat([X_scaled, y], axis=1)).fit()
model_hypertension_revised = smf.ols(formula=formula_hypertension_revised, data=pd.concat([X_scaled, y], axis=1)).fit()
model_diabetes_revised.mse_resid, model_hypertension_revised.mse_resid    

(np.float64(1.0044410698162636), np.float64(5.204203868210578))

In [18]:
model_diabetes_revised.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               diabetes   R-squared:                       0.868
Model:                            OLS   Adj. R-squared:                  0.867
Method:                 Least Squares   F-statistic:                     1106.
Date:                Fri, 02 Jan 2026   Prob (F-statistic):               0.00
Time:                        23:56:34   Log-Likelihood:                -2879.8
No. Observations:                2031   AIC:                             5786.
Df Residuals:                    2018   BIC:                             5859.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                       13.7848      0.179     76.870      0.000      13.433      14.136
C(urbanization_level)[T.2.0]    -0.3319      0.190     -1.749      0.080      -0.704       0.040
C(urbanization_level)[T.3.0]    -0.3244      0.188     -1.727      0.084      -0.693       0.044
C(urbanization_level)[T.4.0]    -0.2933      0.190     -1.547      0.122      -0.665       0.079
C(urbanization_level)[T.5.0]    -0.3041      0.185     -1.639      0.101      -0.668       0.060
C(urbanization_level)[T.6.0]     0.0012      0.185      0.007      0.995      -0.361       0.364
avg_pm25                        -0.1158      0.025     -4.719      0.000      -0.164      -0.068
smoking                          0.5650      0.038     14.793      0.000       0.490       0.640
obesity                          0.3226      0.035      9.116      0.000       0.253       0.392
binge_drinking                  -0.6289      0.030    -20.831      0.000      -0.688      -0.570
lack_of_health_insurance         0.6281      0.043     14.608      0.000       0.544       0.712
routine_checkup                  0.6580      0.029     22.545      0.000       0.601       0.715
lifestyle_insecurity             0.8719      0.058     14.942      0.000       0.757       0.986
==============================================================================
Omnibus:                       43.207   Durbin-Watson:                   1.216
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               91.005
Skew:                          -0.028   Prob(JB):                     1.73e-20
Kurtosis:                       4.035   Cond. No.                         35.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [19]:
model_hypertension_revised.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           hypertension   R-squared:                       0.834
Model:                            OLS   Adj. R-squared:                  0.833
Method:                 Least Squares   F-statistic:                     843.4
Date:                Fri, 02 Jan 2026   Prob (F-statistic):               0.00
Time:                        23:56:34   Log-Likelihood:                -4550.4
No. Observations:                2031   AIC:                             9127.
Df Residuals:                    2018   BIC:                             9200.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                       36.3651      0.408     89.089      0.000      35.565      37.166
C(urbanization_level)[T.2.0]     1.0132      0.432      2.345      0.019       0.166       1.860
C(urbanization_level)[T.3.0]     1.1184      0.428      2.615      0.009       0.280       1.957
C(urbanization_level)[T.4.0]     1.1069      0.432      2.564      0.010       0.260       1.953
C(urbanization_level)[T.5.0]     1.1927      0.422      2.825      0.005       0.365       2.021
C(urbanization_level)[T.6.0]     1.8613      0.421      4.423      0.000       1.036       2.687
avg_pm25                        -0.2403      0.056     -4.302      0.000      -0.350      -0.131
smoking                          1.4377      0.087     16.537      0.000       1.267       1.608
obesity                          0.8737      0.081     10.847      0.000       0.716       1.032
binge_drinking                  -2.1043      0.069    -30.620      0.000      -2.239      -1.970
lack_of_health_insurance         0.3636      0.098      3.715      0.000       0.172       0.556
routine_checkup                  1.9300      0.066     29.050      0.000       1.800       2.060
lifestyle_insecurity             0.3428      0.133      2.581      0.010       0.082       0.603
==============================================================================
Omnibus:                       25.460   Durbin-Watson:                   1.293
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               29.079
Skew:                          -0.215   Prob(JB):                     4.85e-07
Kurtosis:                       3.399   Cond. No.                         35.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The revised regression results indicate that after addressing multicollinearity, the coefficient for `avg_pm25` is still negative. This suggests that other unmeasured confounding factors may be influencing the observed relationships. Further investigation on confounders causing suppression effects will be evaluated in future analyses.